In [9]:
import Pkg;
Pkg.instantiate()
Pkg.activate(".")
import MeshCat as mc
using LinearAlgebra

  Activating environment at `~/Desktop/16745/recitations-2024/16745proj/Project.toml`


In [25]:
# getting rotation matrix from axis-angle vectors
function skew(v)
    [0 -v[3] v[2]; v[3] 0 -v[1]; -v[2] v[1] 0]
end
function dcm_from_phi(ϕ)
    # rotation matrix from axis angle 
    # phi = r * θ, where theta is the axis of rotation (unit vector)
    # and θ = the angle of rotation
    theta = norm(ϕ)
    r = (abs(theta) > 1e-12) ? ϕ / theta : zeros(3)
    Q = (I + sin(theta) * skew(r) + (1.0 - cos(theta)) *
                                    skew(r) * skew(r))
    return Q
end

# create a cylinder from a length and a radius 
function create_cyl(vis, len, R)
    # x axis aligned 
    a = [-len / 2, 0, 0]
    b = [+len / 2, 0, 0]
    cyl = mc.Cylinder(mc.Point(a...), mc.Point(b...), R)
    material = mc.MeshPhongMaterial(color=mc.RGBA(1.0, 0.0, 0.0, 1.0))
    mc.setobject!(vis[:rope], cyl, material)
end

function create_rope(vis, len, R)
    # x axis aligned 
    a = [-len / 2, 0, 0]
    b = [+len / 2, 0, 0]
    cyl = mc.Cylinder(mc.Point(a...), mc.Point(b...), R)
    material = mc.MeshPhongMaterial(color=mc.RGBA(1.0, 0.0, 0.0, 1.0))
    return cyl, material
end

function create_rope(vis, len, R, A, B)
    # x axis aligned 
    a = A
    b = B
    cyl = mc.Cylinder(mc.Point(a...), mc.Point(b...), R)
    material = mc.MeshPhongMaterial(color=mc.RGBA(1.0, 0.0, 0.0, 1.0))
    return cyl, material
end

function get_dcm_from_ends(a, b)
    # get a rotation matrix from ends of a cylinder 
    nx = normalize(b - a)
    e = [1.242, 4.242, -3.424]
    # taking a cross product with random e gives us a new vector 
    # that is orthogonal to both nx and e. We make this ny 
    ny = normalize(cross(nx, e))
    nz = normalize(cross(nx, ny))

    N_Q_B = [nx ny nz]
end

function initialise_quads(N,rope_length,mc,vis)
    quad_obj = mc.MeshFileGeometry(joinpath(@__DIR__, "quadrotor.obj"))
    quad_material = mc.MeshPhongMaterial(color=mc.RGBA(1.0, 0.6, 0.6, 0.7))
    quad_r_offset = [0, 0, 0]
    quad_scale = 2.5
    quad_Q_offset = dcm_from_phi([0, 0, 0])
    quad_offset = mc.compose(
        mc.Translation(quad_r_offset),
        mc.LinearMap(quad_scale * quad_Q_offset)
    )
    
    quad_rotor_position = []
    for i = 1:N
        quad_offset = mc.compose(
            mc.Translation([3 * cos(i * 2 * pi / N), 3 * sin(i * 2pi / N), 0.5]),
            mc.LinearMap(quad_scale * quad_Q_offset)
        )

        mc.setobject!(vis[Symbol("quad", i)][:base], quad_obj, quad_material)
        mc.settransform!(vis[Symbol("quad", i)][:base], quad_offset)

        cyl, material = create_rope(vis, rope_length, 0.05, [3 * cos(i * 2 * pi / N), 3 * sin(i * 2pi / N), 0.5], [0.0,0.0,0.0])
        mc.setobject!(vis[Symbol("rope", i)], cyl, material)
    end
end 

function initialise_ball(mc,vis)
    # dragon details (play with these to see how they change)
dragon_obj = mc.HyperSphere(mc.Point(0.0, 0.0, 0.0), 0.5)
dragon_material = mc.MeshPhongMaterial(color=mc.RGBA(0.6, 0.6, 1.0, 0.7))
dragon_r_offset = [0, 0, 0]
dragon_Q_offset = dcm_from_phi(pi / 2 * [1, 0, 0])
dragon_offset = mc.compose(
    mc.Translation(dragon_r_offset),
    mc.LinearMap(dragon_Q_offset)
)

# create first dragon 
mc.setobject!(vis[:dragon][:base], dragon_obj, dragon_material) # NOTE :base
mc.settransform!(vis[:dragon][:base], dragon_offset)            # NOTE :base
end 

initialise_ball (generic function with 1 method)

In [27]:
let
    vis = mc.Visualizer()

    # NOTICE HOW I USE [:<OBJECT>][:base] WHEN DEFINING THE INITIAL OBJ OFFSETS 
    # THEN WHEN I DO ANIMATION, I JUST USE [:<OBJECT>]


    initialise_ball(mc,vis)
    rope_length = 5.0
    N = 12

    initialise_quads(N,rope_length,mc,vis)
    mc.render(vis)
end

┌ Info: Listening on: 127.0.0.1:8715, thread id: 1
└ @ HTTP.Servers /Users/harshul/.julia/packages/HTTP/vnQzp/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8715
└ @ MeshCat /Users/harshul/.julia/packages/MeshCat/QXID5/src/visualizer.jl:64


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("rope10" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x63, 0x61, 0x74, 0x2f, 0x72, 0x6f, 0x70, 0x65, 0x31, 0x30], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "quad11" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x75, 0x61, 0x64, 0x31, 0x31, 0x2f, 0x62, 0x61, 0x73, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x61, 0x64, 0x31, 0x31, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "rope9" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x72, 0x6f, 0x70, 0x65, 0x39], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rope4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x72, 0x6f, 0x70, 0x65, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "quad10" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x75, 0x61, 0x64, 0x31, 0x30, 0x2f, 0x62, 0x61, 0x73, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x61, 0x64, 0x31, 0x30, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "quad9" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x71, 0x75, 0x61, 0x64, 0x39, 0x2f, 0x62, 0x61, 0x73, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x71, 0x75, 0x61, 0x64, 0x39, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "quad12" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x75, 0x61, 0x64, 0x31, 0x32, 0x2f, 0x62, 0x61, 0x73, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x75, 0x61, 0x64, 0x31, 0x32, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "quad5" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x71, 0x75, 0x61, 0x64, 0x35, 0x2f, 0x62, 0x61, 0x73, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x71, 0x75, 0x61, 0x64, 0x35, 0x2f, 0x62, 0x61, 0x73, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "rope7" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x72, 0x6f, 0x70, 0x65, 0x37], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{Strin